In [1]:
# trying the greedy caption function step-by-step

import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from PIL import Image
import imageio
from skimage.transform import resize

In [2]:
image_path = '/home/gusviloca@GU.GU.SE/aics-vizwiz/data/vizwiz_images/VizWiz_train_00008121.jpg'
model = '/home/gusviloca@GU.GU.SE/aics-vizwiz/code/image-captioning/BEST_checkpoint_vizwiz_5_cap_per_img_5_min_word_freq.pth.tar' 
word_map = '/home/gusviloca@GU.GU.SE/aics-vizwiz/out_data/WORDMAP_vizwiz_5_cap_per_img_5_min_word_freq.json'

In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
#device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print('device', device)

max_len = 50

device cuda:2


In [4]:
# Load model
#checkpoint = torch.load(args.model, map_location=str(device))
checkpoint = torch.load(model, map_location='cuda:2')
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()

Encoder(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 2

In [7]:
# Load word map (word2ix)
with open(word_map, 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

In [8]:
word_map

{'a': 1,
 'can': 2,
 'of': 3,
 'coca': 4,
 'cola': 5,
 'on': 6,
 'counter': 7,
 'is': 8,
 'shown': 9,
 'for': 10,
 'when': 11,
 'one': 12,
 'use': 13,
 'nice': 14,
 'cold': 15,
 'drink': 16,
 'black': 17,
 'zero': 18,
 'calorie': 19,
 'soda': 20,
 'the': 21,
 'near': 22,
 'coffee': 23,
 'maker': 24,
 'kitchen': 25,
 'various': 26,
 'items': 27,
 'top': 28,
 'including': 29,
 'metal': 30,
 'containers': 31,
 'and': 32,
 'tin': 33,
 'placed': 34,
 'surface': 35,
 'with': 36,
 'canisters': 37,
 'kettle': 38,
 'crushed': 39,
 'tomatoes': 40,
 'are': 41,
 'brown': 42,
 'read': 43,
 'brand': 44,
 'sitting': 45,
 'beige': 46,
 'colored': 47,
 'in': 48,
 'puree': 49,
 'from': 50,
 'price': 51,
 'branded': 52,
 'image': 53,
 'view': 54,
 'white': 55,
 'screen': 56,
 'captcha': 57,
 'that': 58,
 'needs': 59,
 'to': 60,
 'be': 61,
 'completed': 62,
 'screenshot': 63,
 'code': 64,
 'phone': 65,
 'smartphone': 66,
 'case': 67,
 'security': 68,
 'measure': 69,
 'shows': 70,
 'page': 71,
 'cell': 72,

In [9]:
rev_word_map

{1: 'a',
 2: 'can',
 3: 'of',
 4: 'coca',
 5: 'cola',
 6: 'on',
 7: 'counter',
 8: 'is',
 9: 'shown',
 10: 'for',
 11: 'when',
 12: 'one',
 13: 'use',
 14: 'nice',
 15: 'cold',
 16: 'drink',
 17: 'black',
 18: 'zero',
 19: 'calorie',
 20: 'soda',
 21: 'the',
 22: 'near',
 23: 'coffee',
 24: 'maker',
 25: 'kitchen',
 26: 'various',
 27: 'items',
 28: 'top',
 29: 'including',
 30: 'metal',
 31: 'containers',
 32: 'and',
 33: 'tin',
 34: 'placed',
 35: 'surface',
 36: 'with',
 37: 'canisters',
 38: 'kettle',
 39: 'crushed',
 40: 'tomatoes',
 41: 'are',
 42: 'brown',
 43: 'read',
 44: 'brand',
 45: 'sitting',
 46: 'beige',
 47: 'colored',
 48: 'in',
 49: 'puree',
 50: 'from',
 51: 'price',
 52: 'branded',
 53: 'image',
 54: 'view',
 55: 'white',
 56: 'screen',
 57: 'captcha',
 58: 'that',
 59: 'needs',
 60: 'to',
 61: 'be',
 62: 'completed',
 63: 'screenshot',
 64: 'code',
 65: 'phone',
 66: 'smartphone',
 67: 'case',
 68: 'security',
 69: 'measure',
 70: 'shows',
 71: 'page',
 72: 'cell',

In [10]:
# get_greedy_caption(encoder, decoder, image_path, word_map):

vocab_size = len(word_map)

# Read image and process
img = imageio.imread(image_path)
if len(img.shape) == 2:
    img = img[:, :, np.newaxis]
    img = np.concatenate([img, img, img], axis=2)
img = resize(img, (256, 256))
img = img.transpose(2, 0, 1)
img = img / 255.
img = torch.FloatTensor(img).to(device)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([normalize])
image = transform(img)  # (3, 256, 256)

# Encode
image = image.unsqueeze(0)  # (1, 3, 256, 256)
encoder_out = encoder(image)  # (1, enc_image_size, enc_image_size, encoder_dim)
enc_image_size = encoder_out.size(1)
encoder_dim = encoder_out.size(3)

In [11]:
# Flatten encoding
encoder_out = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
num_pixels = encoder_out.size(1)

In [12]:
# Start decoding 
h, c = decoder.init_hidden_state(encoder_out)

In [13]:
seq_indices = [] # list of words by its indices
in_text = '<start>'
idx_start_token = 0
inputs = idx_start_token # rename to get previous index

In [15]:
# keys of start and end tokens
start_key = next((k for k in rev_word_map if rev_word_map[k] == '<start>'), None)
end_key = next((k for k in rev_word_map if rev_word_map[k] == '<end>'), None)

print('start', start_key)
print('end', end_key)

start 5930
end 5931


In [16]:
scores = decoder.fc(h)
scores = F.log_softmax(scores, dim=1) # turn logits into probabilities
highest = torch.argmax(scores) # return highest probability
highest = highest.item() # convert from tensor to integer

In [17]:
word = rev_word_map[highest]

In [18]:
word

'down'

In [ ]:
# how to make into a for loop
# what is the model.predict in this case 
    # cos need to feed image and sequence!!!
